In [21]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [2]:
dictNettoyage=pd.read_excel('data/Dict3.xlsx',sheet_name=1)

In [3]:
fichier1=pd.read_excel('data/fichier1.xlsx')
fichier2=pd.read_excel('data/fichier2.xlsx')
fichier3=pd.read_excel('data/fichier3.xlsx')

In [4]:
fichier1.drop(axis=0,inplace=True,index=0)
fichier2.drop(axis=0,inplace=True,index=0)
fichier2.drop(axis=0,inplace=True,index=1)
fichier3.drop(axis=0,inplace=True,index=0)
fichier3.drop(axis=0,inplace=True,index=1)

In [5]:
colonnesNotes=[['Baccalauréat.7','Baccalauréat.8'],['Baccalauréat.9','Baccalauréat.10'],['Baccalauréat.11','Baccalauréat.12']]
matieresNotes=["Mathématiques","Physique-chimie"]
res={}
for m in matieresNotes:
    res[m]=[]
    for col in colonnesNotes:
        nom=col[0]
        note=col[1]
        rows=fichier3[fichier3[col[0]]==m][col[1]]
        for i in rows.index:
            res[m].append([i,note])
colMath='Baccalauréat.14'
colPhys='Baccalauréat.15'
for key in res.keys():
    for couple in res[key]:
        i=couple[0]
        col=couple[1]
        if key == "Mathématiques":
            fichier3.loc[i,colMath]=fichier3.loc[i,col]
        else:
            fichier3.loc[i,colPhys]=fichier3.loc[i,col]

In [6]:
fichier3.drop(columns=[x for l in colonnesNotes for x in l ],inplace=True)

In [7]:
fichier3.to_excel("data/ANOM3.xlsx")

In [8]:
fichiers=[fichier1,fichier2,fichier3]

In [9]:
n=len(fichier1)+len(fichier2)+len(fichier3)
hashMap={}

for row in dictNettoyage.itertuples():
    colFinal=row[1]
    colFichier1=row[2]
    colFichier2=row[3]
    colFichier3=row[4]
    if colFinal=="0":
        continue
    hashMap[colFinal]=[]
    if colFichier1 in fichier1.columns:
        hashMap[colFinal].append(fichier1[colFichier1].astype(str))
    if not colFichier1 in fichier1.columns:
        nan_arr=np.empty(len(fichier1))
        nan_arr[:]=np.nan
        hashMap[colFinal].append(nan_arr)
    if colFichier2 in fichier2.columns:
        hashMap[colFinal].append(fichier2[colFichier2].astype(str))
    if not colFichier2 in fichier2.columns :
        nan_arr=np.empty(len(fichier2))
        nan_arr[:]=np.nan
        hashMap[colFinal].append(nan_arr)
    if colFichier3 in fichier3.columns:
        hashMap[colFinal].append(fichier3[colFichier3].astype(str))
    if not colFichier3 in fichier3.columns :
        nan_arr=np.empty(len(fichier3))
        nan_arr[:]=np.nan
        hashMap[colFinal].append(nan_arr)
    hashMap[colFinal]=[x for l in hashMap[colFinal] for x in l]
    

In [10]:
def compterTypes(data):
    allTypes={}
    float_or_int_regex = r'^-?\d+(\.\d+)?$'
    bool_regex = r'^(True|False)$'
    for col in data.keys():
        keys=np.unique(data[col])
        allTypes[col]=set()
        for key in keys:
            if key == 'nan' or key=="nan":
                continue
            if re.match(float_or_int_regex, str(key)):
                allTypes[col].add("numeric")
            elif re.match(bool_regex, str(key)):
                allTypes[col].add("bool")
            else :
                allTypes[col].add("string")
    return allTypes

In [11]:
def compterAModifiers(data):
    allTypes=compterTypes(data)
    aModifier=[]
    for key in allTypes.keys():
        if len(allTypes[key])>1:
            aModifier.append(key)
    return aModifier

In [12]:
def compterAIntervenir(data):
    aModifier=compterAModifiers(data)
    allTypes=compterTypes(data)
    for key in aModifier:
        if len(allTypes[key])>1 and allTypes[key]!={'string','numeric'}:
            print(key,allTypes[key],np.unique(data[key]))

In [13]:
compterAIntervenir(hashMap)

Genre {'bool', 'string'} ['False' 'True' 'f' 'm' 'o']
Experience_entreprise_1 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']
Experience_entreprise_2 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']
Experience_entreprise_3 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']


In [23]:
listGenre=hashMap['Genre']
for i in range(len(listGenre)):
    if listGenre[i]=='True' or listGenre[i]=='m':
        listGenre[i]=1
    elif listGenre[i]=='False' or listGenre[i]=='f':
        listGenre[i]=0
    elif listGenre[i]=='o':
        listGenre[i]=2
hashMap['Genre']=listGenre

In [24]:
compterAIntervenir(hashMap)

In [25]:
def reparerExperience(l):
    n = len(l)
    for i in range(n):
        if l[i]=='OUI' or l[i]=='oui':
            l[i]='True'
        elif l[i]=='non':
            l[i]='False'
    return l

In [26]:
colonnes=['Experience_entreprise_1','Experience_entreprise_2','Experience_entreprise_3']
for col in colonnes:
    hashMap[col]=reparerExperience(hashMap[col])

In [27]:
compterAIntervenir(hashMap)

In [38]:
allTypes=compterTypes(hashMap)
for key in allTypes.keys():
    if allTypes[key]=={'bool'}:
        hashMap[key]=[bool(x) for x in hashMap[key]]

In [44]:
allTypes=compterTypes(hashMap)
colonnesCorr=[]
for key in allTypes.keys():
    if allTypes[key]=={'bool'} or allTypes[key]=={'numeric'}:
        colonnesCorr.append(key)

In [66]:
res={}
for key in colonnesCorr:
    res[key]=hashMap[key]
dfTotal=pd.DataFrame(res)
dfTotal.to_excel("data/DataTotal.xlsx")

In [107]:
#
#sns.heatmap(matrix, cmap="Greens", annot=True)

In [108]:
matrix=dfTotal.corr()
couplesCorr=[]
cols=matrix.columns.tolist()
colsCorr=set()
for i in range(len(cols)):
    for j in range(i,len(cols)):
        if matrix.iloc[i,j]>=0.5 and i!=j:
            couplesCorr.append([cols[i],cols[j]])
            colsCorr.add(cols[i])
            colsCorr.add(cols[j])

In [106]:
#corr=dfTotal[list(colsCorr)].corr()
#sns.heatmap(corr, cmap="Greens", annot=True)